<a href="https://colab.research.google.com/github/DEFINER24/2025DCC/blob/main/Yolo%EB%AA%A8%EB%8D%B8%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO 모델 파일
## 0. 구글 드라이브 마운트 및 압축 해제

In [ ]:
# ==========================================================
# 0. 구글 드라이브 마운트 및 압축 해제
# ==========================================================
from google.colab import drive
drive.mount('/content/drive')

import zipfile, os
from pathlib import Path

# ZIP 파일 경로 지정
zip_path = '/content/drive/MyDrive/model_data/Model_data.zip'
extract_path = '/content/drive/MyDrive/data_model'

# 압축 해제
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"압축 해제 완료: {extract_path}")
else:
    print("⚠️ Model_data.zip 파일을 찾을 수 없습니다.")


Mounted at /content/drive


In [ ]:
# 압축 해제 후 폴더 구조 점검
import os
for folder in ['train', 'val', 'test']:
    full_path = os.path.join(extract_path, folder)
    if os.path.exists(full_path):
        n_img = len(os.listdir(os.path.join(full_path, 'images')))
        n_lbl = len(os.listdir(os.path.join(full_path, 'labels')))
        print(f"{folder}: images {n_img}개, labels {n_lbl}개")
    else:
        print(f"{folder} 폴더가 존재하지 않습니다.")


train: images 68376개, labels 60034개
val 폴더가 존재하지 않습니다.
test: images 8545개, labels 8545개


## 본 코드

In [ ]:
# ==========================================================
# 0. 환경 설정 및 라이브러리 설치
# ==========================================================
!pip -q install ultralytics==8.3.50
import os, json, glob, random
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import pandas as pd
import numpy as np
import torch
from ultralytics import YOLO

print("Torch:", torch.__version__)
print("GPU available:", torch.cuda.is_available())


In [ ]:
# ==========================================================
# 1. 구글 드라이브 마운트 및 데이터 경로 지정
# ==========================================================
from google.colab import drive
drive.mount('/content/drive')

# 실제 폴더 구조 반영
DATA_ROOT = Path('/content/drive/MyDrive/data_model').resolve()
assert (DATA_ROOT / 'train').exists(), "train 폴더 없음"
assert (DATA_ROOT / 'val').exists(), "val 폴더 없음"
assert (DATA_ROOT / 'test').exists(), "test 폴더 없음"


In [ ]:
# ==========================================================
# 2. 클래스 정의
# ==========================================================
CLASS_NAMES = ['관로', '박스', '맨홀', '패칭', '지하공동']
CLASS_TO_ID = {c:i for i,c in enumerate(CLASS_NAMES)}
print(CLASS_TO_ID)


In [ ]:

# ==========================================================
# 3. JSON → YOLO 포맷 변환 함수
# ==========================================================
def load_image_size(img_path: Path):
    with Image.open(img_path) as im:
        return im.size

def to_yolo_bbox(bx, by, bw, bh, img_w, img_h):
    cx = (bx + bw/2.0) / img_w
    cy = (by + bh/2.0) / img_h
    nw = bw / img_w
    nh = bh / img_h
    return max(0,min(cx,1)), max(0,min(cy,1)), max(0,min(nw,1)), max(0,min(nh,1))

def parse_one_json(jpath: Path, img_dir: Path):
    with open(jpath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    img_name = data.get('fileName')
    candidates = [img_dir / img_name,
                  img_dir / (Path(img_name).stem + '.jpg'),
                  img_dir / (Path(img_name).stem + '.png'),
                  img_dir / (Path(img_name).stem + '.jpeg')]
    img_path = next((c for c in candidates if c.exists()), None)
    if img_path is None:
        raise FileNotFoundError(f"이미지 없음: {img_name}")

    img_w, img_h = load_image_size(img_path)
    ann = data.get('annotation', {})
    anns = ann if isinstance(ann, list) else [ann]

    yolo_lines = []
    for a in anns:
        cls_str = a.get('classes') or data.get('targetType')
        if cls_str not in CLASS_TO_ID:
            continue
        cid = CLASS_TO_ID[cls_str]
        bx, by = float(a['bbox_x']), float(a['bbox_y'])
        bw, bh = float(a['bbox_w']), float(a['bbox_h'])
        if a.get('bbox_xM') and a.get('bbox_yM'):
            bxM, byM = float(a['bbox_xM']), float(a['bbox_yM'])
            bw, bh = max(1.0, bxM - bx), max(1.0, byM - by)
        cx, cy, nw, nh = to_yolo_bbox(bx, by, bw, bh, img_w, img_h)
        yolo_lines.append(f"{cid} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")
    return img_path, yolo_lines, (data.get('plane') or '').upper()


In [ ]:
# ==========================================================
# 4. XZ 평면 데이터만 변환
# ==========================================================
def convert_split(split_name: str):
    split_dir = DATA_ROOT / split_name
    img_dir   = split_dir / 'images'
    json_dir  = split_dir / 'labels'
    yolo_lbl  = split_dir / 'labels_yolo'
    yolo_lbl.mkdir(exist_ok=True)
    json_files = sorted(glob.glob(str(json_dir / '*.json')))
    ok, skip = 0, 0
    per_class_counts = {c:0 for c in CLASS_NAMES}

    for jp in json_files:
        jp = Path(jp)
        try:
            img_path, yolo_lines, plane = parse_one_json(jp, img_dir)
        except Exception as e:
            skip += 1
            continue
        if plane != 'XZ':   # XZ 평면만 사용
            skip += 1
            continue

        out_lbl = yolo_lbl / (img_path.stem + '.txt')
        with open(out_lbl, 'w', encoding='utf-8') as f:
            f.write('\n'.join(yolo_lines))
        ok += 1
        for line in yolo_lines:
            cid = int(line.split()[0])
            per_class_counts[CLASS_NAMES[cid]] += 1
    print(f"[{split_name}] 변환 성공: {ok}, 제외: {skip}")
    print(f"[{split_name}] 클래스 분포:", per_class_counts)
    return ok

convert_split('train')
convert_split('val')
convert_split('test')

In [ ]:
# ==========================================================
# 5. data.yaml 생성
# ==========================================================
YAML_PATH = DATA_ROOT / 'data.yaml'
yaml_text = f"""
path: {DATA_ROOT.as_posix()}
train: train/images
val: val/images
test: test/images
names:
  0: 관로
  1: 박스
  2: 맨홀
  3: 패칭
  4: 지하공동
"""
with open(YAML_PATH, 'w', encoding='utf-8') as f:
    f.write(yaml_text)
print(YAML_PATH.read_text())


In [ ]:
# ==========================================================
# 6. YOLO 모델 학습 (XZ 평면 데이터만)
# ==========================================================
MODEL_NAME = 'yolov8m.pt'
IMG_SIZE   = 896
EPOCHS     = 100
BATCH      = 16

model = YOLO(MODEL_NAME)
results = model.train(
    data=str(YAML_PATH),
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(DATA_ROOT / 'runs'),
    name='xz_only_yolo',
    verbose=True
)

best_ckpt = Path(results.save_dir) / 'weights' / 'best.pt'
print("최적 가중치 경로:", best_ckpt)


In [ ]:
# ==========================================================
# 7. 검증 및 mAP@0.5 계산
# ==========================================================
best_model = YOLO(best_ckpt)
val_res = best_model.val(
    data=str(YAML_PATH),
    split='test',
    imgsz=IMG_SIZE,
    device=0 if torch.cuda.is_available() else 'cpu',
    conf=0.001,
    iou=0.5,
    max_det=3000
)
mAP50 = float(val_res.results_dict.get('metrics/mAP50', np.nan))
print(f"mAP@IoU=0.5: {mAP50:.4f}")


In [ ]:

# ==========================================================
# 8. mAP@IoU=0.5 그래프 시각화
# ==========================================================
val_dir = val_res.save_dir
csv_path = os.path.join(val_dir, 'results.csv')
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    key = [c for c in df.columns if 'mAP50' in c][0]
    plt.figure(figsize=(8,5))
    plt.plot(df[key], label='mAP@0.5', color='blue', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('mAP@IoU=0.5')
    plt.title('Epoch-wise mAP@0.5 (XZ plane only)')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print("results.csv 파일이 존재하지 않습니다.")
